In [6]:
pip install timm

In [ ]:
# from transformers import AutoTokenizer, AutoModel, CLIPImageProcessor
# from PIL import Image
# import torch

# path = "OpenGVLab/InternVL-Chat-V1-1"
# model = AutoModel.from_pretrained(
#     path,
#     torch_dtype=torch.bfloat16,
#     low_cpu_mem_usage=True,
#     use_flash_attn=True,
#     trust_remote_code=True).eval().cuda()
# tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)



Unrecognized keys in `rope_scaling` for 'rope_type'='dynamic': {'type'}
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL-Chat-V1-1:
- modeling_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


FlashAttention2 is not installed.


pytorch_model.bin.index.json:   0%|          | 0.00/90.5k [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/619k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/254 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: 'https://m.media-amazon.com/images/I/81dzao1Ob4L.jpg'

In [ ]:
# image_processor = CLIPImageProcessor.from_pretrained(path)
# image1 = Image.open('/content/616BH+J+rwS.jpg').resize((448, 448))
# pixel_values1 = image_processor(images=image1, return_tensors='pt').pixel_values.to(torch.bfloat16).cuda()
# image2 = Image.open('/content/51fuXO81VkL.jpg').resize((448, 448))
# pixel_values2 = image_processor(images=image2, return_tensors='pt').pixel_values.to(torch.bfloat16).cuda()
# pixel_values = torch.cat((pixel_values1, pixel_values2), dim=0)
# num_patches_list = [pixel_values1.size(0), pixel_values2.size(0)]

# generation_config = dict(max_new_tokens=1024, do_sample=False)
# questions = ['<image>\nfind the width of the product in the image'] * len(num_patches_list)
# responses = model.batch_chat(tokenizer, pixel_values,
#                              num_patches_list=num_patches_list,
#                              questions=questions,
#                              generation_config=generation_config)
# for question, response in zip(questions, responses):
#     print(f'User: {question}')
#     print(f'Assistant: {response}')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


User: <image>
find the width of the product in the image
Assistant: The width of the product in the image is 2.1 inches.
User: <image>
find the width of the product in the image
Assistant: The width of the product in the image is 76cm.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os
import re
import requests
import pandas as pd
from PIL import Image
from transformers import AutoTokenizer, AutoModel, CLIPImageProcessor
import torch

# Load the model and tokenizer
path = "OpenGVLab/InternVL2-8B"
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)
image_processor = CLIPImageProcessor.from_pretrained(path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


FlashAttention2 is not installed.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
import pandas as pd
import requests
from PIL import Image
import torch
import os
import re

# Define entity unit map
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Function to standardize unit
def standardize_unit(unit):
    unit_map = {
        'cm': 'centimetre', 'm': 'metre', 'mm': 'millimetre', 'ft': 'foot', 'in': 'inch', 'yd': 'yard', 'inches': 'inch','"':'inch',
        'g': 'gram', 'kg': 'kilogram', 'mg': 'milligram', 'mcg': 'microgram', 'oz': 'ounce', 'lb': 'pound', 'tonne': 'ton',
        'v': 'volt', 'kv': 'kilovolt', 'mv': 'millivolt', 'w': 'watt', 'kw': 'kilowatt', 'l': 'litre', 'ml': 'millilitre',
        'cl': 'centilitre', 'dl': 'decilitre', 'fl oz': 'fluid ounce', 'gal': 'gallon', 'imp gal': 'imperial gallon',
        'cu in': 'cubic inch', 'cu ft': 'cubic foot', 'pt': 'pint', 'qt': 'quart'
    }
    return unit_map.get(unit.lower(), unit)

# Function to extract value and unit from text
def extract_value_and_unit(text, entity_name):
    # Define regex pattern for extracting value and unit
    pattern = r'(\d+\.?\d*)\s*([a-zA-Z]+)'
    matches = re.findall(pattern, text)

    for match in matches:
        value, unit = match
        unit = standardize_unit(unit)
        if unit in entity_unit_map[entity_name]:
            return value, unit
    return None, None

# Function to process the CSV and prepare for download
def process_csv(input_csv_path, output_csv_path):
    df = pd.read_csv(input_csv_path)
    entity_values = []

    for index, row in df.iterrows():
        image_url = row['image_link']
        entity_name = row['entity_name']

        # Download the image
        image_path = f"{index}.jpg"
        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)

        # Open and preprocess the image
        image = Image.open(image_path).resize((448, 448))
        pixel_values = image_processor(images=image, return_tensors='pt').pixel_values.to(torch.bfloat16).cuda()

        # Ask the LLM the question about the entity
        questions = [f'<image>\n answer in one line :Analyse the product in this image, and write the most appropriate height of the object present otherwise say none']
        responses = model.batch_chat(tokenizer, pixel_values, num_patches_list=[pixel_values.size(0)],
                                     questions=questions, generation_config={'max_new_tokens': 1024, 'do_sample': False})

        # Extract value and unit
        response_text = responses[0]
        print(response_text)
        value, unit = extract_value_and_unit(response_text, entity_name)

        # Append result to list
        if value and unit:
            entity_values.append(f"{value} {unit}")
        else:
            entity_values.append(None)

        # Print link and output for verification
        print(f"Image URL: {image_url} | Extracted Output: {entity_values[-1]}")

        # Clean up the downloaded image
        os.remove(image_path)

    # Add the entity values to a new column
    df['entity_value'] = entity_values

    # Save processed CSV to the specified output file path
    df.to_csv(output_csv_path, index=False)

    return output_csv_path



In [ ]:
from google.colab import files

# Call the function to process the CSV file
processed_csv_path = process_csv('/content/drive/MyDrive/amzn-data/height/height_batch_1.csv', '/content/height_1.csv')

# Provide a link to download the processed CSV in Colab
files.download(processed_csv_path)


Streaming output truncated to the last 5000 lines.
The most appropriate height of the object present in the image is 13.1 cm.
Image URL: https://m.media-amazon.com/images/I/41bLd5-kfML.jpg | Extracted Output: 13.1 centimetre
The most appropriate height of the object present in the image is 30.3 inches.
Image URL: https://m.media-amazon.com/images/I/41bMQJhydIL.jpg | Extracted Output: 30.3 inch
The most appropriate height of the object present is 10.7 cm or 4.2 inches.
Image URL: https://m.media-amazon.com/images/I/41bNTT925ML.jpg | Extracted Output: 10.7 centimetre
The most appropriate height of the object present in the image is 74 cm (29.1 inches).
Image URL: https://m.media-amazon.com/images/I/41bO8tKedOL.jpg | Extracted Output: 74 centimetre
The most appropriate height of the object present in the image is 17.5 cm.
Image URL: https://m.media-amazon.com/images/I/41bOTnuTKyL.jpg | Extracted Output: 17.5 centimetre
The most appropriate height of the object present is 1.02 inches.
Imag

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
processed_csv_path = process_csv('/content/drive/MyDrive/amzn-data/height/height_batch_2.csv', '/content/height_2.csv')




Streaming output truncated to the last 5000 lines.
The most appropriate height of the object present in the image is 9 cm.
Image URL: https://m.media-amazon.com/images/I/51BO72h1uXL.jpg | Extracted Output: 9 centimetre
The most appropriate height of the object present in the image is 4.72 inches.
Image URL: https://m.media-amazon.com/images/I/51BO7VWvPSL.jpg | Extracted Output: 4.72 inch
The most appropriate height of the object present in the image is 76 cm.
Image URL: https://m.media-amazon.com/images/I/51BOHBrD76S.jpg | Extracted Output: 76 centimetre
The most appropriate height of the object present in the image is 114mm.
Image URL: https://m.media-amazon.com/images/I/51BOIaU3U0L.jpg | Extracted Output: 114 millimetre
The most appropriate height of the object present in the image is 167 cm.
Image URL: https://m.media-amazon.com/images/I/51BOWu9eWiL.jpg | Extracted Output: 167 centimetre
The most appropriate height of the object present in the image is 8.66 inches (22 cm).
Image URL

In [ ]:
from google.colab import files
# Provide a link to download the processed CSV in Colab
files.download(processed_csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
processed_csv_path = process_csv('/content/drive/MyDrive/amzn-data/height/height_batch_3.csv', '/content/height_3.csv')



In [ ]:
from google.colab import files

# Provide a link to download the processed CSV in Colab
files.download(processed_csv_path)

In [ ]:
processed_csv_path = process_csv('/content/drive/MyDrive/amzn-data/height/height_batch_4.csv', '/content/height_4.csv')


In [ ]:
from google.colab import files

# Provide a link to download the processed CSV in Colab
files.download(processed_csv_path)

In [ ]:
processed_csv_path = process_csv('/content/drive/MyDrive/amzn-data/height/height_batch_5.csv', '/content/heigh_5.csv')



In [ ]:
from google.colab import files

# Provide a link to download the processed CSV in Colab
files.download(processed_csv_path)

In [ ]:
processed_csv_path = process_csv('/content/drive/MyDrive/amzn-data/height/height_batch_6.csv', '/content/height_6.csv')



In [ ]:
from google.colab import files

# Provide a link to download the processed CSV in Colab
# Provide a link to download the processed CSV in Colab
files.download(processed_csv_path)

In [ ]:
processed_csv_path = process_csv('/content/drive/MyDrive/amzn-data/height/height_batch_7.csv', '/content/height_7.csv')



The most appropriate height of the object present in the image is 180 mm.
Image URL: https://m.media-amazon.com/images/I/61CSVUsyrcL.jpg | Extracted Output: 180 millimetre
The most appropriate height of the object present in the image is 32.5 inches.
Image URL: https://m.media-amazon.com/images/I/61CSZKVbiBL.jpg | Extracted Output: 32.5 inch
The most appropriate height of the object present in the image is 71.75 inches (70.5 cm).
Image URL: https://m.media-amazon.com/images/I/61CSaCQYK9L.jpg | Extracted Output: 71.75 inch
The most appropriate height of the object present is 30 cm.
Image URL: https://m.media-amazon.com/images/I/61CSazQU6nL.jpg | Extracted Output: 30 centimetre
The most appropriate height of the object present in the image is 7.25 inches.
Image URL: https://m.media-amazon.com/images/I/61CSeZjrH9L.jpg | Extracted Output: 7.25 inch
The most appropriate height of the object present is 80 cm.
Image URL: https://m.media-amazon.com/images/I/61CSmHGfLgL.jpg | Extracted Output: 

In [ ]:
from google.colab import files

# Provide a link to download the processed CSV in Colab
# Provide a link to download the processed CSV in Colab
files.download(processed_csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
import pandas as pd
import requests
from PIL import Image
import torch
import os
import re

# Define entity unit map
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Function to standardize unit
def standardize_unit(unit):
    unit_map = {
        'cm': 'centimetre', 'm': 'metre', 'mm': 'millimetre', 'ft': 'foot', 'in': 'inch', 'yd': 'yard', 'inches': 'inch','"':'inch',
        'g': 'gram', 'kg': 'kilogram', 'mg': 'milligram', 'mcg': 'microgram', 'oz': 'ounce', 'lb': 'pound', 'tonne': 'ton',
        'v': 'volt', 'kv': 'kilovolt', 'mv': 'millivolt', 'w': 'watt', 'kw': 'kilowatt', 'l': 'litre', 'ml': 'millilitre',
        'cl': 'centilitre', 'dl': 'decilitre', 'fl oz': 'fluid ounce', 'gal': 'gallon', 'imp gal': 'imperial gallon',
        'cu in': 'cubic inch', 'cu ft': 'cubic foot', 'pt': 'pint', 'qt': 'quart', "volts":"volt", "vac": "volt"
    }
    return unit_map.get(unit.lower(), unit)

# Function to extract value and unit from text
def extract_value_and_unit(text, entity_name):
    # Define regex pattern for extracting value and unit
    pattern = r'(\d+\.?\d*)\s*([a-zA-Z]+)'
    matches = re.findall(pattern, text)

    for match in matches:
        value, unit = match
        unit = standardize_unit(unit)
        if unit in entity_unit_map[entity_name]:
            return value, unit
    return None, None

# Function to process the CSV and prepare for download
def process_csv(input_csv_path, output_csv_path):
    df = pd.read_csv(input_csv_path)
    entity_values = []

    for index, row in df.iterrows():
        image_url = row['image_link']
        entity_name = row['entity_name']

        # Download the image
        image_path = f"{index}.jpg"
        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)

        # Open and preprocess the image
        image = Image.open(image_path).resize((448, 448))
        pixel_values = image_processor(images=image, return_tensors='pt').pixel_values.to(torch.bfloat16).cuda()

        # Ask the LLM the question about the entity
        questions = [f'<image>\n answer in one line :find the voltage of the product in the image if found otherwise say none']
        responses = model.batch_chat(tokenizer, pixel_values, num_patches_list=[pixel_values.size(0)],
                                     questions=questions, generation_config={'max_new_tokens': 1024, 'do_sample': False})

        # Extract value and unit
        response_text = responses[0]
        print(response_text)
        value, unit = extract_value_and_unit(response_text, entity_name)

        # Append result to list
        if value and unit:
            entity_values.append(f"{value} {unit}")
        else:
            entity_values.append(None)

        # Print link and output for verification
        print(f"Image URL: {image_url} | Extracted Output: {entity_values[-1]}")

        # Clean up the downloaded image
        os.remove(image_path)

    # Add the entity values to a new column
    df['entity_value'] = entity_values

    # Save processed CSV to the specified output file path
    df.to_csv(output_csv_path, index=False)

    return output_csv_path



In [23]:
processed_csv_path = process_csv('/content/voltage.csv', '/content/voltage_mod.csv')



Streaming output truncated to the last 5000 lines.
none
Image URL: https://m.media-amazon.com/images/I/51QdywYiAYL.jpg | Extracted Output: None
None
Image URL: https://m.media-amazon.com/images/I/51QfJ4x7avL.jpg | Extracted Output: None
The voltage of the product in the image is 100-240V.
Image URL: https://m.media-amazon.com/images/I/51QgOu-qzdL.jpg | Extracted Output: 240 volt
none
Image URL: https://m.media-amazon.com/images/I/51QhrZF9+9L.jpg | Extracted Output: None
None
Image URL: https://m.media-amazon.com/images/I/51Qi-zvaF+L.jpg | Extracted Output: None
The voltage of the product in the image is 120 Volts.
Image URL: https://m.media-amazon.com/images/I/51QiQx38EAL.jpg | Extracted Output: 120 volt
The voltage of the product in the image is 4.7V.
Image URL: https://m.media-amazon.com/images/I/51QixDKRmxL.jpg | Extracted Output: 4.7 volt
None
Image URL: https://m.media-amazon.com/images/I/51Qj4ZIMkSL.jpg | Extracted Output: None
The voltage of the product in the image is 12V.
Imag

In [10]:
import pandas as pd
import requests
from PIL import Image
import torch
import os
import re

# Define entity unit map
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Function to standardize unit
def standardize_unit(unit):
    unit_map = {
        'cm': 'centimetre', 'm': 'metre', 'mm': 'millimetre', 'ft': 'foot', 'in': 'inch', 'yd': 'yard', 'inches': 'inch','"':'inch',
        'g': 'gram', 'kg': 'kilogram', 'mg': 'milligram', 'mcg': 'microgram', 'oz': 'ounce', 'lb': 'pound', 'tonne': 'ton',
        'v': 'volt', 'kv': 'kilovolt', 'mv': 'millivolt', 'w': 'watt', 'kw': 'kilowatt', 'l': 'litre', 'ml': 'millilitre',
        'cl': 'centilitre', 'dl': 'decilitre', 'fl': 'fluid ounce', 'gal': 'gallon', 'imp gal': 'imperial gallon',
        'cu in': 'cubic inch', 'cu ft': 'cubic foot', 'pt': 'pint', 'qt': 'quart', "volts":"volt", "vac": "volt"
    }
    return unit_map.get(unit.lower(), unit)

# Function to extract value and unit from text
def extract_value_and_unit(text, entity_name):
    # Define regex pattern for extracting value and unit
    pattern = r'(\d+\.?\d*)\s*([a-zA-Z]+)'
    matches = re.findall(pattern, text)

    for match in matches:
        value, unit = match
        unit = standardize_unit(unit)
        if unit in entity_unit_map[entity_name]:
            return value, unit
    return None, None

# Function to process the CSV and prepare for download
def process_csv(input_csv_path, output_csv_path):
    df = pd.read_csv(input_csv_path)
    entity_values = []

    for index, row in df.iterrows():
        image_url = row['image_link']
        entity_name = row['entity_name']

        # Download the image
        image_path = f"{index}.jpg"
        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)

        # Open and preprocess the image
        image = Image.open(image_path).resize((448, 448))
        pixel_values = image_processor(images=image, return_tensors='pt').pixel_values.to(torch.bfloat16).cuda()

        # Ask the LLM the question about the entity
        questions = [f'<image>\n answer in one line :find the volume of the object in image is mentioned, otherwise say none(if the unit is fl oz write it as fl_oz in your reply)']
        responses = model.batch_chat(tokenizer, pixel_values, num_patches_list=[pixel_values.size(0)],
                                     questions=questions, generation_config={'max_new_tokens': 1024, 'do_sample': False})

        # Extract value and unit
        response_text = responses[0]
        print(response_text)
        value, unit = extract_value_and_unit(response_text, entity_name)

        # Append result to list
        if value and unit:
            entity_values.append(f"{value} {unit}")
        else:
            entity_values.append(None)

        # Print link and output for verification
        print(f"Image URL: {image_url} | Extracted Output: {entity_values[-1]}")

        # Clean up the downloaded image
        os.remove(image_path)

    # Add the entity values to a new column
    df['entity_value'] = entity_values

    # Save processed CSV to the specified output file path
    df.to_csv(output_csv_path, index=False)

    return output_csv_path



In [11]:
processed_csv_path = process_csv('/content/item_volume.csv', '/content/item_volume_mod.csv')



15 ml
Image URL: https://m.media-amazon.com/images/I/21DZ7BAZ6-L.jpg | Extracted Output: 15 millilitre
2 fl_oz
Image URL: https://m.media-amazon.com/images/I/31-Jdk3SMUL.jpg | Extracted Output: 2 fluid ounce
The volume of the object in the image is 20/20 fl_oz.
Image URL: https://m.media-amazon.com/images/I/310uxz1C1+L.jpg | Extracted Output: 20 fluid ounce


KeyboardInterrupt: 

In [15]:
import pandas as pd
import requests
from PIL import Image
import torch
import os
import re

# Define entity unit map
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Function to standardize unit
def standardize_unit(unit):
    unit_map = {
        'cm': 'centimetre', 'm': 'metre', 'mm': 'millimetre', 'ft': 'foot', 'in': 'inch', 'yd': 'yard', 'inches': 'inch','"':'inch',
        'g': 'gram', 'kg': 'kilogram', 'mg': 'milligram', 'mcg': 'microgram', 'oz': 'ounce', 'lb': 'pound', 'tonne': 'ton',
        'v': 'volt', 'kv': 'kilovolt', 'mv': 'millivolt', 'w': 'watt', 'kw': 'kilowatt', 'l': 'litre', 'ml': 'millilitre',
        'cl': 'centilitre', 'dl': 'decilitre', 'fl oz': 'fluid ounce', 'gal': 'gallon', 'imp gal': 'imperial gallon',
        'cu in': 'cubic inch', 'cu ft': 'cubic foot', 'pt': 'pint', 'qt': 'quart'
    }
    return unit_map.get(unit.lower(), unit)

# Function to extract value and unit from text
def extract_value_and_unit(text, entity_name):
    # Define regex pattern for extracting value and unit
    pattern = r'(\d+\.?\d*)\s*([a-zA-Z]+)'
    matches = re.findall(pattern, text)

    for match in matches:
        value, unit = match
        unit = standardize_unit(unit)
        if unit in entity_unit_map[entity_name]:
            return value, unit
    return None, None

# Function to process the CSV and prepare for download
def process_csv(input_csv_path, output_csv_path):
    df = pd.read_csv(input_csv_path)
    entity_values = []

    for index, row in df.iterrows():
        image_url = row['image_link']
        entity_name = row['entity_name']

        # Download the image
        image_path = f"{index}.jpg"
        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)

        # Open and preprocess the image
        image = Image.open(image_path).resize((448, 448))
        pixel_values = image_processor(images=image, return_tensors='pt').pixel_values.to(torch.bfloat16).cuda()

        # Ask the LLM the question about the entity
        questions = [f'<image>\n answer in one line : does the image looks like a container? if so what is the height mentioned? if not a container, just say no(mention height only if container)']
        responses = model.batch_chat(tokenizer, pixel_values, num_patches_list=[pixel_values.size(0)],
                                     questions=questions, generation_config={'max_new_tokens': 1024, 'do_sample': False})

        # Extract value and unit
        response_text = responses[0]
        print(response_text)
        value, unit = extract_value_and_unit(response_text, entity_name)

        # Append result to list
        if value and unit:
            entity_values.append(f"{value} {unit}")
        else:
            entity_values.append(None)

        # Print link and output for verification
        print(f"Image URL: {image_url} | Extracted Output: {entity_values[-1]}")

        # Clean up the downloaded image
        os.remove(image_path)

    # Add the entity values to a new column
    df['entity_value'] = entity_values

    # Save processed CSV to the specified output file path
    df.to_csv(output_csv_path, index=False)

    return output_csv_path







In [13]:
import pandas as pd

def split_csv(input_file, batch_size):
  """
  Splits a CSV file into multiple batches.

  Args:
    input_file: Path to the input CSV file.
    batch_size: Size of each batch.
  """
  df = pd.read_csv(input_file)
  num_batches = len(df) // batch_size + (len(df) % batch_size > 0)
  for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(df))
    batch_df = df[start_idx:end_idx]
    output_file = f"/content/depth_batch_3_split_{i+1}.csv"
    batch_df.to_csv(output_file, index=False)
    print(f"Saved batch {i+1} to {output_file}")

# Split the CSV file into batches of 1005 rows
split_csv('/content/drive/MyDrive/amzn-data/depth/depth_batch_3.csv', 1005)


Saved batch 1 to /content/depth_batch_3_split_1.csv
Saved batch 2 to /content/depth_batch_3_split_2.csv
Saved batch 3 to /content/depth_batch_3_split_3.csv
Saved batch 4 to /content/depth_batch_3_split_4.csv
Saved batch 5 to /content/depth_batch_3_split_5.csv


In [ ]:

# Assuming the split files are named depth_batch_1_split_*.csv
import glob

# Get a list of all split CSV files
split_files = glob.glob('/content/depth_batch_3_split_*.csv')

# Process each split file
for file in split_files:
  # Extract the batch number from the filename
  batch_number = file.split('_')[-1].split('.')[0]
  output_file = f'/content/depth_{batch_number}.csv'

  # Process the CSV file
  processed_csv_path = process_csv(file, output_file)

  # Download the processed CSV
  files.download(processed_csv_path)



No, the image does not look like a container.
Image URL: https://m.media-amazon.com/images/I/51elgCHuL9L.jpg | Extracted Output: None
No, the image does not look like a container. The height mentioned is 42 cm.
Image URL: https://m.media-amazon.com/images/I/51elkaWTaqL.jpg | Extracted Output: 42 centimetre
No, the image does not look like a container. The height mentioned is 320 mm.
Image URL: https://m.media-amazon.com/images/I/51elsWxWjXL.jpg | Extracted Output: 320 millimetre
Yes, the image looks like a container. The height mentioned is 6.5 inches (16.7 cm).
Image URL: https://m.media-amazon.com/images/I/51elu2tMyXL.jpg | Extracted Output: 6.5 inch
Yes, the image looks like a container. The height mentioned is 3.5 inches (88mm).
Image URL: https://m.media-amazon.com/images/I/51emK+nC-OL.jpg | Extracted Output: 3.5 inch
Yes, the image looks like a container. The height mentioned is 93 cm.
Image URL: https://m.media-amazon.com/images/I/51en6xP59nL.jpg | Extracted Output: 93 centimetr